## Install deepchem

### Colab

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

In [ ]:
!pip install --pre deepchem-nightly==2.3.0
import deepchem
deepchem.__version__

In [ ]:
!pip install tensorflow-determinism

In [ ]:
import tensorflow


device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

tensorflow.__version__

### Locally

Follow instructions here to setup your conda env with deepchem: https://deepchem.readthedocs.io/en/latest/installation.html#conda-installation

Additionally install requirements:

pip install -r requirements.txt

## Initialize

In [ ]:
import itertools
import numpy as np
from dataclasses import dataclass
from sklearn.ensemble import RandomForestRegressor
from typing import Union, List, Tuple, NoReturn, Optional
import random
import os

import deepchem as dc
from deepchem.models import KerasModel
from tqdm import tqdm

ROOT_DIR = "/content/drive/My Drive/Colab Notebooks/deep learning for the life sciences/chapter_5_protein_binding"
DATA_DIR = os.path.join(ROOT_DIR, "data")

## Load the data

### Cached data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pathlib import Path
import os

print(os.listdir(DATA_DIR))
print(os.listdir('/content/drive'))

In [ ]:
# for unmounting
# from google.colab import drive
# drive.flush_and_unmount()

In [ ]:
pdbbind_tasks, pdbbind_datasets, transformers = dc.molnet.load_pdbbind(
    featurizer="grid", split="random", subset="core", data_dir=DATA_DIR, save_dir=os.path.join(DATA_DIR, "from-pdbbind"), split_seed=100, reload=True
)
train_dataset, valid_dataset, test_dataset = pdbbind_datasets

In [ ]:
@dataclass
class OutputModel:
    model: Union[dc.models.Model, dc.models.sklearn_models.SklearnModel]
    layers: Optional[List[int]]
    dropout: Optional[Union[float, List[float]]]
    seed: int
    train_score: Optional[float]
    test_score: Optional[float]


def create_nn_model(
    layer_sizes: List[int], dropouts: Union[float, List[float]], learning_rate: float = 0.0003
) -> dc.models.Model:
    n_features = train_dataset.X.shape[1]
    model = dc.models.MultitaskRegressor(
        n_tasks=len(pdbbind_tasks),
        n_features=n_features,
        layer_sizes=layer_sizes,
        dropouts=dropouts,
        learning_rate=learning_rate,
        model_dir=f"{ROOT_DIR}/model_dumps/pdbbind_nn__{'-'.join(map(str, layer_sizes))}_dropout_{dropouts}_learningrate_{learning_rate}",
    )
    return model


def create_rf_model(number_of_trees_in_the_forest: int, seed: int) -> dc.models.Model:
    sklearn_model = RandomForestRegressor(n_estimators=number_of_trees_in_the_forest, random_state=seed)
    model = dc.models.SklearnModel(sklearn_model)  
    return model


def evaluate_model(model: dc.models.Model) -> Tuple[float, float]:
    metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
    train_scores = model.evaluate(train_dataset, [metric], transformers)["pearson_r2_score"]
    test_scores = model.evaluate(test_dataset, [metric], transformers)["pearson_r2_score"]
    return train_scores, test_scores


def visualize_model_output(model: OutputModel) -> NoReturn:
    print(
        f"""############################################################
Visualizing {model.model.__class__.__name__} model
  Training score: {model.train_score}
  Test score: {model.test_score}
  Hyperparameters: 
    Seed: {model.seed}
    Layers: {model.layers} 
    Dropout: {model.dropout}

Model summary:
""")
    if issubclass(type(model.model), dc.models.KerasModel):
        model.model.model.summary()
    elif issubclass(type(model.model), dc.models.sklearn_models.SklearnModel):
      print(model.model.model_instance)
    print("############################################################")

In [ ]:
layers_list = [
    [125, 62],
    # [250, 125],
    # [500, 250],
    # [1000, 500],
    # [2000, 1000],
    # [4000, 2000],
    # [2000, 1000, 500, 250],
]

dropouts_list = [0.8]

seeds = [0, 0]

hyperparameters = list(itertools.product(layers_list, dropouts_list, seeds))

(layers_list, dropouts_list, seeds)

In [ ]:
nn_models = []
rf_models = []

In [ ]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISM'] = '1'

for i in tqdm(range(len(hyperparameters))):
    layers, dropout, seed = hyperparameters[i]
    # random.seed(seed)
    # np.random.RandomState(seed)
    model_nn = create_nn_model(layer_sizes=layers, dropouts=dropout)
    model_nn.fit(train_dataset, nb_epoch=50)
    train_score, test_score = evaluate_model(model=model_nn)
    nn_models.append(
        OutputModel(
            model=model_nn,
            layers=layers,
            dropout=dropout,
            seed=seed,
            train_score=train_score,
            test_score=test_score,
        )
    )

In [ ]:
for seed in seeds:
    model_rf = create_rf_model(100, seed)
    model_rf.fit(train_dataset)
    train_score, test_score = evaluate_model(model=model_rf)
    rf_models.append(
        OutputModel(
              model=model_rf,
              layers=None,
              dropout=None,
              seed=seed,
              train_score=train_score,
              test_score=test_score,
          )
    )
    print(f"""
    Seed: {seed}
      Training score: {train_score}
      Test score: {test_score}
    """)

In [ ]:
for model in nn_models:
    visualize_model_output(model)

In [ ]:
for model in rf_models:
    visualize_model_output(model)

In [ ]:
!python --version